##### 저장된 모델 활용
- 모델 파일 종류
    - 가중치 및 절편 저장 파일=> 동일한 구조 모델 인스턴스 생성 후 사용 가능
    - 모델 전체 저장 파일=> 바로 사용 가능

[1] 모듈 로딩<hr>

In [6]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 처리
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *

In [7]:
# 모델 파라미터(가중치와 절편) 저장
import os
# 경로
SAVE_PATH= '../models/iris/'
# 이름
SAVE_FILE='model_train_wbs.pth'
# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL='model_all.pth'

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)       #하위 폴더까지 생성

In [8]:
class IrisRegModel(nn.Module):
    
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self) -> None:
        super().__init__()
        self.in_layer= nn.Linear(3, 10)
        self.hidden_layer= nn.Linear(10, 30)
        self.out_layer= nn.Linear(30, 1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y= self.in_layer(input_data)    #변수가 3개 절편이 하나인 식이 퍼셉트론당 1개씩 나옴
        y= F.relu(y)                   #0 ~ y 사이의 값 도출

        # 은닉층
        y= self.hidden_layer(y)         #10개 입력 => 30개 출력
        y= F.relu(y)                   # 0~ y 사이의 값

        # 출력층
        y= self.out_layer(y)            #30개 입력 => 1개 출력 (petal_width 최종 예측값)
        return y

In [9]:
# 학습 진행 관련 설정값

EPOCH= 20
BATCH_SIZE= 10  #버리는 데이터 없게 하려면 전체 데이터 수를 보고 조절
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
LR= 0.001

[2] 모델 로딩 (전체 파일 사용)

In [11]:
irisModel=torch.load(SAVE_MODEL, weights_only=False)

[3] 예측<hr>

In [12]:
data=[float(x) for x in input("SL, SW, PL").split(',')]
data= torch.FloatTensor(data).reshape(1,-1)
data.shape

torch.Size([1, 3])

In [ ]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF) -> None:
        super().__init__()
        self.featureDF= featureDF
        self.targetDF= targetDF
        self.n_rows= featureDF.shape[0]
        self.n_features= featureDF.shape[1]


    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화 (전결합층=> 기본이 FloatTensor)
        featureTS= torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [ ]:
# 데이터셋 인스턴스
X_train, X_test, Y_train, Y_test= train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, Y_train, Y_val= train_test_split(X_train, Y_train, random_state=1)
print(X_train.shape, X_test.shape, X_val.shape)

irisDS= IrisDataset(X_train, Y_train) #검증용 데이터 클래스로 만들기 싫으면 나중에 텐서로 변환후 넣으면 됨
valDS=IrisDataset(X_val, Y_val)
testDS= IrisDataset(X_test, Y_test)

In [13]:
# 새로운 데이터에 대한 예측, 즉 predict
irisModel.eval()
with torch.no_grad():

        # 학습/추론론
        pre_val=irisModel(data)

        # 손실

In [14]:
pre_val

tensor([[0.6839]])